# 구문 분석: Part III - RDP, SR, and Left-Corner Parser

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 8.Analyzing sentence structure(https://www.nltk.org/book/ch08.html) 내용을 참고해서 자료를 만듦. 
    - nltk book의 8장 예제를 일부 사용.
- 참고 서적
    - Natural Language Processing with Python Cookbook, Krishna Bhavsar, Naresh Kumar, Pratap Dangeti, Packt Publishing, 2017.
    - Chapter 6, Chapter 7의 예제를 일부 사용.

### Prerequisite
동영상 강의(6장-하향식 구문분석 및 상향식 구문분석)을 시청한 후에 실습을 진행해야 함.

여기서 다룰 내용은
- 다양한 문장 구조를 표현하려면 형식 문법을 어떻게 사용하면 될까? 
- 구문 트리를 사용하여 문장 구조를 어떻게 보여줄 수 있을까?
- 파서가 문장을 어떻게 분석하고, 구문 트리를 자동으로 만들어 낼까?

In [ ]:
import nltk 

# 3.   CFG 구문 분석

- 2개의 파싱 알고리즘, top-down 방식(recursive descent parsing)과 bottom-up 방식(shift-reduce parsing)에 대해 알아본다. 
- bottom-up filtering이 적용된 top-down 방식(left-corner parsing)과 동적 프로그래밍 방식(chart parsing)에 대해서도 알아본다.

## 3.1   Recursive Descent Parsing (RDP)
- 가장 간단한 파서인 RDP는 문법을 규정(specification)으로 해석한다. 
    - RDP는 문법을 상위 레벨 목표를 여러 개의 하위 레벨 목표로 나누어 표현한 것으로 해석한다.
        - 최상위 레벨 목표는 시작 기호 *S*를 찾는 것이다. 
        - 파서는 **S → NP VP** 규칙을 보고, 두 개의 하위 레벨 목표(*NP*를 찾고나서 *VP*를 찾음)로 나눈다. 
    - 하위 레벨 목표는 다시 *NP* 및 *VP*에 대한 하위-하위 레벨 목표로 나뉜다. 
        - **NP -> Det Nom | PropN** 
        - **VP -> V Adj | V NP | V S | V NP PP** 
    - 이러한 확장 과정은 최하위 레벨 목표까지 이어지며, 마침내 terminal 기호(*Mary, saw, a , dog*)를 만나게 된다.
        - terminal 기호는 입력 문장의 token과 직접 비교할 수 있다.
        - 두 단어를 비교해서 같으면 다음 목표로 전환되지만, 
        - 같지 않으면 backtracking을 통해 지금껏 구성했던 트리를 지우고 다른 대안을 선택한다.
    - RDP는 이 과정을 진행하면서 구문 트리를 구성한다.

In [ ]:
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
""")

Recursive descent parsing(RDP)은 3가지 중요한 단점이 있다. 
- 1. **NP-> NP PP** 와 같은 좌순환 생성규칙(left recursive rule)이 있으면 무한 루프에 빠진다.
- 2. 생성규칙에 나타난 순서대로 조사하기 때문에, 입력 문장과 일치하지 않는 단어에 대해서도 조사하느라 시간을 낭비한다. 
- 3. backtracking이 발생하면 이제껏 만들었던 구문 트리를 지우고 이전 상태로 되돌아가야 한다.

RDP 알고리즘은 top-down 방식의 일종이다. 
- top-down 파서는 입력이 어떤가 들여다 보고 결정하는 것이 아니라 
    - 문법에서 정의한 생성규칙 순서대로 하나씩 생성 규칙과 입력을 비교해 나가는 방식이다. 
- 입력 정보를 미리 들여다 보고 구문 분석하는 것이 훨씬 더 효과적인데, 그 방식이 바로 bottom-up 방식이다. 

In [ ]:
rd_parser = nltk.RecursiveDescentParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in rd_parser.parse(sent):
    print(tree)
    tree.draw()

위에서 진행한 과정을 메소드로 구현하면서 파싱 구현을 단계별로 복습해 보자.

In [ ]:
def rd_parser(grammar, textlist):
    parser = nltk.RecursiveDescentParser(grammar)
    for text in textlist:
        sentence = nltk.word_tokenize(text)
        for tree in parser.parse(sentence):
            print(tree)
            tree.draw()

In [ ]:
grammar2 = nltk.CFG.fromstring("""
S -> NP VP
NP -> NNP VBZ
VP -> IN NNP | DT NN IN NNP
NNP -> 'Incheon' | 'Songdo' | 'Seoul' | 'Korea'
VBZ -> 'is'
IN -> 'in' | 'of'
DT -> 'the'
NN -> 'capital'
""")

In [ ]:
text = [
    "Songdo is in Incheon",
    "Seoul is the capital of Korea"
]

In [ ]:
rd_parser(grammar2, text)

## 3.2   Shift-Reduce Parsing

- NLTK는 shift-reduce parser를 구현한  ShiftReduceParser()를 제공한다. 
- 이 파서는 backtracking을 전혀 실행하지 않기 때문에 주어진 입력에 대해 제대로 된 구문 분석 결과를 생성한다는 보장이 없다. 
- 또한, 여러 개의 구문 분석 트리가 존재하더라도 오직 한 개의 구문 트리만을 찾는다. 

- shift-reduce 파서는 입력 문장이 문법에 맞더라도 dead end에 도달하여 제대로 파싱 못할 수도 있다. 
    - 이런 상황이 발생하면, 입력 버퍼는 비었지만, parsing stack에는 시작 기호 *S*로 reduce할 수 없는 기호들이 남아 있다. 
    - 이런 문제가 발생한 이유는 파서의 선택이 잘못되었지만 이를 취소할 수 없었기 때문이다. 
    - 파서는 reduce-reduce conflict와 shift-reduct conflict 등 2가지 형태의 선택에 직면한다. 
- shift-reduce 파서는 이러한 충돌을 회피하기 위한 정책을 구현하고 있다. 
    - 즉 shift-reduce conflict가 발생하면 어떤 reduction도 가능하지 않을 경우 shift 연산으로 이를 해결한다. 
    - 또, reduce-reduce 충돌이 발생하면 가장 빈번히 적용한 reduction을 우선 적용해 충돌을 해결하는 방안이다. 
- 가장 대표적인 shift-reduce parser는 **lookahead LR 파서**이다.
- RDP parser에 비해 shift-reduceSR parser의 장점은 입력 단어에 해당하는 구조만 만든다는 것이다. 
    - 또한 하위 구조도 한 번만 만든다.

In [ ]:
sr_parser = nltk.ShiftReduceParser(grammar1, trace=2)
sr_parser.grammar()

In [ ]:
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
    print(tree)

- ShiftReduceParser 생성자 중 2번째 파라미터 **trace**는 tracing output의 레벨을 설정.
    - 타입: int
    - trace level 
        - 0 : tracing output을 생성하지 않음.
        - 1 : shift action만을 출력.
        - 2:  shift와 reduce action을 모두 출력.
        - 3: 어느 token이 shift되고, 어느 production이 reduce되는가를 출력. 

In [ ]:
sr_parser = nltk.ShiftReduceParser(grammar2, trace=2)
sent = 'Songdo is in Incheon'.split()
for tree in sr_parser.parse(sent):
    print(tree)

shift reduce 파서가 파싱하는 과정을 단계별로 확인할 수 있음.

In [ ]:
nltk.app.srparser_app.app()

### shift reduce parser와 recursive descent parser의 차이점 비교

In [ ]:
grammar3 = nltk.CFG.fromstring("""
 S -> NP VP
 PP -> P NP
 NP -> 'the' N | N PP | 'the' N PP
 VP -> V NP | V PP | V NP PP
 N -> 'cat'
 N -> 'dog'
 N -> 'rug'
 V -> 'chased'
 V -> 'sat'
 P -> 'in'
 P -> 'on'
 """)

sentence1 = 'the cat chased the dog'.split() #unambiguous sentence
sentence2 = 'the cat chased the dog on the rug'.split() #ambiguous sentence

In [ ]:
sr = nltk.ShiftReduceParser(grammar3)

sentence1은 모호하지 않은 문장이기 때문에 shift reduct parser는 정확히 구문분석을 수행한다.

In [ ]:
for t in sr.parse(sentence1):
     print(t)

- SR parser는 shift-reduct 충돌 또는 reduce-reduce 충돌이 발생하면 경험적 방법으로 충돌을 해결하려고 시도한다.
- sentence2은 모호한 문장이기 때문에 shift reduct parser는 경험적 방법으로 문제를 해결하려고 시도했지만,
    - 불행히도 이러한 경험적 방법이 해결책이 되지는 못했다. 결국 아무런 출력도 생성하지 못했다.

In [ ]:
for t in sr.parse(sentence2):
     print(t)

반면 rd parser는 2가지 가능성 있는 구문 분석 결과를 모두 출력한다.

In [ ]:
rd = RecursiveDescentParser(grammar3)

In [ ]:
for t in rd.parse(sentence1):
     print(t)

In [ ]:
for t in rd.parse(sentence2):
     print(t)

## 3.3   The Left-Corner Parser
- left-corner parser는 bottom-up 방식과 top-down 방식을 결합한 것이다. 즉 bottom-up filtering이 적용된 top-down parser이다.
- left-corner parser는 RDP와 달리 좌순환 생성 규칙으로 인해 무한루프에 빠지지 않는다. 
- "John saw Mary"란 문장을 구문 분석해 보자. **S  -> NP VP**에서 *NP*는 다음 3가지 중 하나로 확장할 수 있다.
    - a.	NP -> Det N
    - b.	NP -> Det N PP
    - c.	NP -> "John" | "Mary" | "Bob"
- 위 3가지 중 가장 먼저 적용해야 할 규칙은 문장의 첫 번째 단어가 'John'이기 때문에 당연히 (c)이다. 
- **A ⇒* B α**일 때, *B*를 루트가 *A*인 트리의 left-corner라고 부른다. left-corner parser는 파생 결과를 미리 예측해 생성 규칙을 선택한다.
- left-corner parser는 문법의 전처리를 통해 테이블을 만든다. 
    - 테이블의 각 행은 2개의 셀을 갖는다. 
    - 첫 번째 셀은 nonterminal 기호를 갖고 있으며, 두 번째 셀은 이 nonterminal 기호의 left-corner 에 속하는 기호들을 갖고 있다. 

In [ ]:
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
""")

### left-corners(pre-terminals)
# S - NP
# VP - V
# PP - P
# NP - Det

sent = 'John saw Mary'.split()

In [ ]:
#print(grammar1)

In [ ]:
from nltk.parse.chart import LeftCornerChartParser

lc_parser = LeftCornerChartParser(grammar1, trace=2)
for tree in lc_parser.parse(sent):
    print(tree)

- Edit > Edit Text 선택 : "John ate the cake on the table"를 "John ate the cake"로 수정 후 OK버튼 클릭.
- Edit > Edit Grammar 선택 : Productions 확인.
- chartparser 중  Bottom Up Left-Corner Predict Rule을 선택. 
    - Bottom Up Left-Corner Predict Rule 버튼을 클릭할 때마다
    - LeftCornerParser의 구문 분석이 진행되는 과정을 단계별로 확인할 수 있음.

In [ ]:
nltk.app.chartparser_app.app()